<a href="https://colab.research.google.com/github/BrunoKrinski/fcn_colab/blob/master/CNN_presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Activate GPU enviroment: Runtime -> Change runtime type -> Hardware accelerator -> GPU

In [0]:
# Select Tensorflow 2.0
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
# Import tensorflow and keras
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Import keras layers

# Conv2D => Convolutional Layer
# Flatten => Convert the 2D feature map of the convolutional step in a 1D vector to input the fully connected layers
# Dense => Fully connected layers
# Dropout => Ignore some neurons to add randomness to the CNN. It helps the CNN to learn the same thing with different neurons
# Activation => Activation layer (relu, softmax, sigmoid,...)
# MaxPooling2D => MaxPooling operation
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Activation, MaxPooling2D
from tensorflow.keras import Sequential, Model

In [0]:
# Import and load a dataset
mnist = tf.keras.datasets.mnist

# x_train => train images
# y_train => train labels
# x_test => test images
# y_test => test labels
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the images to float
x_train, x_test = x_train / 255.0, x_test / 255.0

class_names = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine']

In [0]:
# Show informations about the dataset
print('-----------------------')
print(x_train.shape)
print(len(y_train))
print(y_train)
print('-----------------------')
print(x_test.shape)
print(len(y_test))
print(y_test)
print('-----------------------')

In [0]:
# Show examples of the dataset
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.xlabel(class_names[y_train[i]])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
plt.show()

keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

---

keras.layers.Activation(activation)

---

keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)

---

keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

---

keras.layers.Dropout(rate, noise_shape=None, seed=None)


In [0]:
# Define a sequential CNN model
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [0]:
# Show details of the model
model.summary()

In [0]:
train_images = x_train[..., tf.newaxis]
train_labels = y_train
test_images = x_test[..., tf.newaxis]
test_labels = y_test

print(train_images.shape)
print(test_images.shape)

# Train the model with Adam optimizer, sparse categorical crossentropy loss, and evaluate with accuracy
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with 5 epochs
model.fit(train_images, train_labels, epochs=5)

In [0]:
# Evaluate the model
results = model.evaluate(test_images,  test_labels, verbose=2)
print('test loss, test acc:', results)

In [0]:
# Get a test image and predict it with the trained model
test_image = test_images[245]

plt.figure()
plt.imshow(np.squeeze(test_image,axis=2),cmap=plt.cm.binary)
plt.grid(False)
plt.show()

predictions = model.predict_classes(np.expand_dims(test_image,axis=0))
print('Prediction:',predictions)

In [0]:
# Import pre-trained model with the imagenet weights
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

vgg16 = VGG16(weights='imagenet')

# Show network information
vgg16.summary()

In [0]:
!wget https://assets.nrdc.org/sites/default/files/styles/full_content/public/media-uploads/wlds43_654640_2400.jpg?itok=2eXbSQ8S -O image.jpg

In [0]:
img_path = '/content/image.jpg'

from tensorflow.keras.preprocessing import image
img = image.load_img(img_path, target_size=(224, 224))

plt.figure()
plt.imshow(img)
plt.grid(False)
plt.show()

In [0]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = vgg16.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [0]:
# Import pre-trained models without the dense layers.
# Only the convolutional layers are imported, and can be used to extract features
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

base_model.summary()

In [0]:
# Without the dense step, the model can be adapted to classify other problems
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
  layer.trainable = False

model.summary()